# 8. OpenAI Responses Target

In this demo, we show an example of the `OpenAIResponseTarget`. [Responses](https://platform.openai.com/docs/api-reference/responses) is a newer protocol than chat completions and provides additional functionality with a somewhat modified API. The allowed input types include text, image, web search, file search, functions, reasoning, and computer use.

Before you begin, ensure you are set up with the correct version of PyRIT installed and have secrets configured as described [here](../../setup/populating_secrets.md).
## OpenAI Configuration

Like most targets, all `OpenAITarget`s need an `endpoint` and often also needs a `model` and a `key`. These can be passed into the constructor or configured with environment variables (or in .env).

- endpoint: The API endpoint (`OPENAI_RESPONSES_ENDPOINT` environment variable). For OpenAI, these are just "https://api.openai.com/v1/responses".
- auth: The API key for authentication (`OPENAI_RESPONSES_KEY` environment variable).
- model_name: The model to use (`OPENAI_RESPONSES_MODEL` environment variable). For OpenAI, these are any available model name and are listed here: "https://platform.openai.com/docs/models".

In [ ]:
from pyrit.common import IN_MEMORY, initialize_pyrit
from pyrit.orchestrator import PromptSendingOrchestrator
from pyrit.prompt_target import OpenAIResponseTarget

initialize_pyrit(memory_db_type=IN_MEMORY)

target = OpenAIResponseTarget()

orchestrator = PromptSendingOrchestrator(objective_target=target)

response = await orchestrator.run_attack_async(objective="Tell me a joke")  # type: ignore
await response.print_conversation_async()  # type: ignore

In [1]:
import logging
from pyrit.common import IN_MEMORY, initialize_pyrit
from pyrit.models import PromptRequestPiece, PromptRequestResponse
from pyrit.prompt_target.openai.openai_response_target import OpenAIResponseTarget

# Enable debug logs
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

initialize_pyrit(memory_db_type=IN_MEMORY)

# Tool implementation 
async def get_current_weather(args):
    logger.debug("get_current_weather called with args=%s", args)
    return {
        "weather": "Sunny",
        "temp_c": 22,
        "location": args["location"],
        "unit": args["unit"],
    }

# Responses API function tool schema (flat, no nested "function" key)
function_tool = {
    "type": "function",
    "name": "get_current_weather",
    "description": "Get the current weather in a given location",
    "parameters": {
        "type": "object",
        "properties": {
            "location": {"type": "string", "description": "City and state"},
            "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
        },
        "required": ["location", "unit"],
        "additionalProperties": False,
    },
    "strict": True,
}

# Let the model auto-select tools
target = OpenAIResponseTarget(
    model_name="o4-mini",
    py_functions={"get_current_weather": get_current_weather},
    extra_body_parameters={
        "tools": [function_tool],
        # no tool_choice -> model decides
    },
    httpx_client_kwargs={"timeout": 60.0},
    api_version=None,  # using api.openai.com
)

# Build the user prompt
prompt_piece = PromptRequestPiece(
    role="user",
    original_value="What is the weather in Boston in celsius? Use the get_current_weather function.",
    original_value_data_type="text",
)
prompt_request = PromptRequestResponse(request_pieces=[prompt_piece])

# Run the agentic loop (Jupyter supports `await` directly)
response = await target.send_prompt_async(prompt_request=prompt_request)

for idx, piece in enumerate(response.request_pieces):
    print(f"{idx} | {piece.role}: {piece.original_value}")


INFO:pyrit.common.initialization:Loaded C:\Users\vkuta\projects\PyRIT\.env
INFO:dotenv.main:python-dotenv could not find configuration file .env.
INFO:pyrit.common.initialization:Using in-memory DuckDB database.
INFO:pyrit.memory.duckdb_memory:Engine created successfully for database: :memory:
INFO:pyrit.memory.central_memory:Central memory instance set to: DuckDBMemory
INFO:pyrit.memory.central_memory:Using existing memory instance: DuckDBMemory
INFO:pyrit.prompt_target.openai.openai_chat_target_base:Sending the following prompt to the prompt target: {}: user: What is the weather in Boston in celsius? Use the get_current_weather function.
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x000002AA495D7200>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x000002AA495

0 | assistant: The current weather in Boston, MA is Sunny with a temperature of 22°C.


In [2]:
from pyrit.common import IN_MEMORY, initialize_pyrit
from pyrit.models import PromptRequestPiece, PromptRequestResponse
from pyrit.prompt_target.openai.openai_response_target import OpenAIResponseTarget

initialize_pyrit(memory_db_type=IN_MEMORY)

# This is the OpenAI "web_search" tool config. No function registration is needed for built-in tools.
web_search_tool = {
    "type": "web_search_preview"
    }

target = OpenAIResponseTarget(
    extra_body_parameters={
        "tools": [web_search_tool],
        "tool_choice": "auto",
    },
    httpx_client_kwargs={"timeout": 60} 
)

prompt_piece = PromptRequestPiece(
    role="user",
    original_value="What is a positive news story from today?",
    original_value_data_type="text"
)
prompt_request = PromptRequestResponse(
    request_pieces=[prompt_piece]
)

response = await target.send_prompt_async(prompt_request=prompt_request)

for idx, piece in enumerate(response.request_pieces):
    print(f"{idx} | {piece.role}: {piece.original_value}")


INFO:pyrit.common.initialization:Loaded C:\Users\vkuta\projects\PyRIT\.env
INFO:dotenv.main:python-dotenv could not find configuration file .env.
INFO:pyrit.common.initialization:Using in-memory DuckDB database.
INFO:pyrit.memory.central_memory:Central memory instance set to: DuckDBMemory
INFO:pyrit.memory.central_memory:Using existing memory instance: DuckDBMemory
INFO:pyrit.prompt_target.openai.openai_chat_target_base:Sending the following prompt to the prompt target: {}: user: What is a positive news story from today?
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.anyio.AnyIOStream object at 0x000002AA129789E0>
DEBUG:httpcore.connection:start_tls.started ssl_context=<ssl.SSLContext object at 0x000002AA4960FB50> server_hostname='api.openai.com' timeout=60
DEBUG:httpcore.connection:start_tls.complete return_value=<httpcore._bac

0 | assistant: {"id":"ws_68af1cea4fac8190b519f48a94e834fe0b358e68a696def6","type":"web_search_call","status":"completed","action":{"type":"search","query":"positive news stories today"}}
1 | assistant: Here are some uplifting news stories from August 27, 2025:

**Saturn's Rings Offer Stunning Views**

For the first time in 15 years, Saturn's rings are perfectly tilted, providing a spectacular view of their southern face. This rare alignment is visible through basic telescopes, making it an ideal opportunity for both seasoned astronomers and first-time stargazers. Additionally, Venus and Jupiter appear side-by-side in the morning sky, and the Perseid meteor shower is peaking, offering a celestial showcase. ([podego.com](https://www.podego.com/insights/august-2025-good-news-ai-pfas-stories?utm_source=openai))

**Breakthrough in PFAS Detoxification**

Scientists at the University of Cambridge have discovered that specific gut bacteria can absorb and help eliminate PFAS, the "forever chemi